In [17]:
import pandas as pd
import numpy as np
import glob
import os
from matplotlib import pyplot as plt
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait 
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import heapq
nltk.download("punkt")
nltk.download("stopwords")
stopwords = stopwords.words('english')


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('disable-infobars')
#chrome_options.add_argument('headless')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument('--disable-popup-blocking')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/edenshrian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/edenshrian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
class sceince_direct_text_extractor:
    
    def __init__(self,key_word):
        
        self.key_word = key_word
        self.urls = []
        self.urls_to_drop = []
        self.text_dict = {}
    
    def text_extractor(self):
        
        self.urls = sceince_direct_text_extractor.get_urls(self.key_word)
        self.urls_to_drop , self.text_dict = sceince_direct_text_extractor.get_dict_of_abstract_text(self.urls)
        
        return 
                
    def get_urls(key_word):

        driver = webdriver.Chrome(executable_path= '/Users/edenshrian/Desktop/Documents/Eden Shrian/Eden/Chrome Driver/chromedriver', options=chrome_options)
        driver.get("https://www.sciencedirect.com/")
        driver.maximize_window()
        driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div[1]/div/div/div/div/div/form/div/div[1]/div/div[1]/div/div/input").send_keys(key_word)
        driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div[1]/div/div/div/div/div/form/div/div[1]/div/div[1]/div/div/input").send_keys(Keys.RETURN)
        time.sleep(2.0)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2.0)
        index_of_num_3 = 2
        max_search_in_web = 25
        list_for_iteration = np.delete(np.arange(1,max_search_in_web+2),[index_of_num_3])
        urls = [driver.find_element_by_xpath(f"/html/body/div[1]/div/div/div/div[1]/div/div/section/div/div[2]/main/div[1]/div[2]/div[2]/div/ol/li[{num}]/div/div/h2/span/a").get_attribute("href") for num in list_for_iteration]
        driver.close()

        return urls
    
    def get_unique_examples():
        
        df_urls = pd.read_csv("/Users/edenshrian/Desktop/Documents/Eden Shrian/Eden/Projects/Text Summary/urls.csv")
        unique_examples = list(set(list(df_urls['URLS'].values)))

        return unique_examples

    def parse_page_text(url):

        driver = webdriver.Chrome(executable_path= '/Users/edenshrian/Desktop/Documents/Eden Shrian/Eden/Chrome Driver/chromedriver', options=chrome_options)
        driver.get(url)
        driver.maximize_window()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        title = driver.title
        title = title.replace("-","")
        title = title.replace("ScienceDirect","")
        unique_examples = sceince_direct_text_extractor.get_unique_examples()

        for example in unique_examples:
            try:
                text = driver.find_element_by_xpath(example).text
            except:
                pass

        driver.close()

        return title,text

    def get_dict_of_abstract_text(urls):

        specialChars = '!#$%^&*<>/•'
        dictionary = {}
        urls_to_drop = []
        for url in urls:
            try:
                title,text = sceince_direct_text_extractor.parse_page_text(url)
                for char in specialChars:
                    text = text.replace(char,'')
                dictionary[title] = text.replace("\n","")
            except:
                urls_to_drop = url

        return urls_to_drop,dictionary

In [19]:
class summarize_sceince_direct_abstract:
    
    def __init__(self):
        
        
        self.key_word = 0
        self.num_of_sentences = 0
        self.summary = 0
         
    def get_summary(self,key_word,num_of_sentences):
        
        self.key_word = key_word
        self.num_of_sentences = num_of_sentences
        self.summary = summarize_sceince_direct_abstract.summarize_info_from_sceince_direct(self.key_word,
                                                                                            self.num_of_sentences)
        
        return self.summary

    def create_one_long_text(key_word):
    
        SD_text = sceince_direct_text_extractor(key_word)
        SD_text.text_extractor()
        long_text = "".join(list(SD_text.text_dict.values()))

        return long_text

    def tokenize_words_to_flat_list(long_text):

        text_tokenize = sent_tokenize(long_text)
        words = [word_tokenize(sent_token) for sent_token in text_tokenize] 
        flat_word_list = [item for sublist in words for item in sublist]

        return text_tokenize,flat_word_list

    def create_freq_dict(flat_word_list):

        freq = {}
        for word in flat_word_list:
            if word not in stopwords and word.isalpha():
                if word not in freq.keys():
                    freq[word]  = 1
                else:
                    freq[word] +=1

        return freq

    def calculate_freq_in_dict(freq_dict):

        max_freq = max(freq_dict.values())
        for word in freq_dict.keys():
            freq_dict[word] = (freq_dict[word]/max_freq)

        return freq_dict

    def calculate_sentence_score(text_tokenize,flat_word_list,freq_dict):

        sentence_score = {}
        for sent_token in text_tokenize:
            for word in flat_word_list:
                if word.lower() in freq_dict.keys():
                    if sent_token not in sentence_score.keys():
                        sentence_score[sent_token] = freq_dict[word]
                    else:
                        sentence_score[sent_token] += freq_dict[word]

        return sentence_score

    def get_summary_text(sentence_score,num_of_sentences):

        summary_sentences = heapq.nlargest(num_of_sentences,
                                          sentence_score,
                                          key = sentence_score.get)

        summary = ' '.join(summary_sentences)

        return summary

    def summarize_info_from_sceince_direct(key_word,num_of_sentences):

        long_text = summarize_sceince_direct_abstract.create_one_long_text(key_word)
        text_tokenize,flat_word_list = summarize_sceince_direct_abstract.tokenize_words_to_flat_list(long_text)
        freq_dict = summarize_sceince_direct_abstract.create_freq_dict(flat_word_list)
        sentence_score = summarize_sceince_direct_abstract.calculate_sentence_score(text_tokenize,flat_word_list,freq_dict)
        summary = summarize_sceince_direct_abstract.get_summary_text(sentence_score,num_of_sentences)

        return summary

In [29]:
Summarize = summarize_sceince_direct_abstract()

In [30]:
summary1 = Summarize.get_summary("Deep Learning for stock predictions",3)

In [31]:
summary1

'Stock market prediction has been a classical yet challenging problem, with the attention from both economists and computer scientists. With the purpose of building an effective prediction model, both linear and machine learning tools have been explored for the past couple of decades. Lately, deep learning models have been introduced as new frontiers for this topic and the rapid development is too fast to catch up.'

In [32]:
summary2 = Summarize.get_summary("E-government",3)

In [33]:
summary2

'While much has changed regarding e-government adoption and use by cities and their residents, it remains difficult to contextualize these changes and what they might mean for the development of e-government overall. Adopting e-government services is an innovative practice and there are a number of reasons why local governments might be expected to be particularly slow when it comes to innovation. Most cities have limited communications budgets and are severely restricted as to the amount of resources that could be utilized for potential upgrades.'